In [33]:
import pandas as pd
import numpy as np

# Tema: Análise do algoritmo PageRank
## Autor: Luiz Fernando da Silva
Neste laboratório analizarei o algoritmo PageRank da google aplicando o mesmo a uma base de dados de de links de invesidores de bitcoins, onde cada investidor recebe uma nota de acordo com outras pessoas que fizeram tranzações com eles. Após a aplicação do algoritmo as seguites perguntas serão respondidas.

* Quantas iterações o PageRank precisou rodar até atingir convergência?
* Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?
* Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

In [34]:
data = pd.read_csv('data/soc-sign-bitcoinotc.csv')

# Filtrando dados
Filtrando avaliações que possuem notas maiores ou iguais a 8.

In [35]:
filtered_data = data.query('Evaluation >= 8')
source = list(filtered_data.Source)
target = list(filtered_data.Target)
nodes = list(set(source).union(set(target)))

# Gerando matrix de adjacência
Cada chava da matrix é um id de um usuário e o valor é a lista dos usuários que ele avaliou.

In [36]:
adjacence_dict = {no:[] for no in nodes}

for index in range(len(source)):
    ori = source[index]
    dest = target[index]
    
    adjacence_dict[ori].append(dest)

# Gerando matrix de transição

In [37]:
a = [[0 for i in range(len(nodes))] for j in range(len(nodes))]

for i in range(len(nodes)):
    for j in range(len(nodes)):
        ori = nodes[i]
        dest = nodes[j]
        out_degree = len(adjacence_dict[ori])
        a[j][i] = 1/out_degree if dest in adjacence_dict[ori] else 0

a = np.matrix(a)

In [38]:
b = (1/len(nodes)) * np.matrix([[1 for i in range(len(nodes))] for j in range(len(nodes))])

# Google Matrix
Google matrix com fator pulo 0.15

In [39]:
m = 0.85 * a + 0.15 * b

# Vetor normalizado

In [40]:
v = (1/len(nodes)) * np.matrix([[1] for i in range(len(nodes))])

# Função para calcular o pagerank
Este algoritmo calcula o page rank de cada nó de acordo com a fórmula: $PR(A) = \frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)}... \Rightarrow PR(u) = \sum_{v \in B_u} \frac{PR(B(v))}{L(v)}$, onde PR(...) é o pagerank de um dado nó, L(...) é a quantidade de links que saem do nó, A, B, C, D, u, são nós quaisquer de um grafo e B(v) é um nó do grafo que tem link com o nó u.

In [41]:
count = 0

def pagerank(v):
    global count
    count += 1
    
    if sum(abs(m*v-v)) > 0.001:
        return pagerank(m*v)
    return m*v

# Calculando pagerank
Usando o método de calcular o pagerank para gerar pagerank de cada nó da base de dados filtrada.

In [42]:
result = pagerank(v)
result = [cell.item(0,0) for cell in result]

# Número de iterações necessárias para gerar o pagerank

In [43]:
print(count)

35


# Criando data frame
Criando data frame com o resultado do pagerank de cada nó para posteriormente salvar em um arquivo csv.

In [44]:
result_data_frame = pd.DataFrame({'id': nodes, 'PageRank': result})
result_data_frame.sort_values('PageRank', ascending=False).take(range(5))

,id,PageRank
0,1,0.000139
120,202,0.000125
94,144,0.000117
887,3996,0.000093
182,361,0.000092


# Salvando resultados
Salvando data frame de resultados e o data frame de nós filtrados em um arquivo csv.

In [45]:
result_data_frame.to_csv(path_or_buf='results/result.csv', index=False)
filtered_data.to_csv(path_or_buf='results/filtered-sign-bitcoinotc.csv', columns=['Source', 'Target'], index=False)

# Análise dos resultados

* **Quantas iterações o PageRank precisou rodar até atingir convergência?**
> Foram necessárias 35 iterações até o algoritmo atingir a converegência.
* **Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?**

|  id   | PageRanck |
|:-----:|:---------:|
|   1   | 0.000139  |
|  202  | 0.000125  |
|  144  | 0.000117  |
| 3996  | 0.000093  |
|  361  | 0.000092  |

* **Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?**
> Usando os resultados obtidos pelo pagerank é possível analisar o risco de uma tranação com determinado investidor ser fraudulenta ou não
